In [8]:
from ortools.linear_solver import pywraplp

# Create the solver
solver = pywraplp.Solver.CreateSolver('SCIP')

# Time periods (weeks)
weeks = list(range(1, 9))  # Weeks 1 to 8
next_week_demand = {'P': 35, 'H': 58}  # Demand for week 9 to calculate safety stock

# Time periods (weeks)
weeks = list(range(1, 9))  # Weeks 1 to 8

# Demand data for each week (P-Head, H-Head)
demand_data = [
    (55, 38), (55, 38), (44, 30), (0, 0), (45, 48), (45, 48), (36, 58), (35, 57)
]

# Create a dictionary to hold demand data for P-Heads and H-Heads
demands = {}
for week, (p_demand, h_demand) in zip(weeks, demand_data):
    demands[(week, 'P')] = p_demand
    demands[(week, 'H')] = h_demand

# Production capacities
capacities = {'P': 100, 'H': 80}

# Costs
prod_cost = {'P': 225, 'H': 310}
changeover_cost = 500
annual_inv_cost_rate = 0.195
weekly_inv_cost_rate = annual_inv_cost_rate / 52

# Variables
production = {(week, type): solver.IntVar(0, capacities[type], f'prod_{week}_{type}') for week in weeks for type in ['P', 'H']}
inventory = {(week, type): solver.IntVar(0, solver.infinity(), f'inv_{week}_{type}') for week in weeks for type in ['P', 'H']}
changeover = {(week): solver.BoolVar(f'changeover_{week}') for week in weeks if week > 1}

# Initial inventory
initial_inventory = {'P': 125, 'H': 143}

# Objective: Minimize total cost
total_cost = solver.Objective()
for week in weeks:
    for type in ['P', 'H']:
        # Production cost
        total_cost.SetCoefficient(production[(week, type)], prod_cost[type])
        # Inventory cost
        total_cost.SetCoefficient(inventory[(week, type)], weekly_inv_cost_rate * prod_cost[type])

for week in weeks[1:]:
    # Changeover cost
    total_cost.SetCoefficient(changeover[week], changeover_cost)

total_cost.SetMinimization()

# Constraints
for week in weeks:
    for type in ['P', 'H']:
        # Balance equation: inventory
        if week == 1:
            solver.Add(inventory[(week, type)] == initial_inventory[type] + production[(week, type)] - demands[(week, type)])
        else:
            solver.Add(inventory[(week, type)] == inventory[(week - 1, type)] + production[(week, type)] - demands[(week, type)])
        # Safety stock
        if week < 8:
            solver.Add(inventory[(week, type)] >= 0.8 * demands[(week + 1, type)])
        else:
            solver.Add(inventory[(week, type)] >= 0.8 * next_week_demand[type])

# Solve the model
status = solver.Solve()

# Check the result
if status == pywraplp.Solver.OPTIMAL:
    print("Solution:")
    for week in weeks:
        print(f"Week {week}:")
        for type in ['P', 'H']:
            print(f"  {type}-Head Production: {production[(week, type)].solution_value()} units")
            print(f"  {type}-Head Inventory: {inventory[(week, type)].solution_value()} units")
    print(f"Total cost: {solver.Objective().Value()}")
else:
    print("The problem does not have an optimal solution.")


Solution:
Week 1:
  P-Head Production: 0.0 units
  P-Head Inventory: 70.0 units
  H-Head Production: 0.0 units
  H-Head Inventory: 105.0 units
Week 2:
  P-Head Production: 21.0 units
  P-Head Inventory: 36.0 units
  H-Head Production: 0.0 units
  H-Head Inventory: 67.0 units
Week 3:
  P-Head Production: 8.0 units
  P-Head Inventory: 0.0 units
  H-Head Production: 0.0 units
  H-Head Inventory: 37.0 units
Week 4:
  P-Head Production: 36.0 units
  P-Head Inventory: 36.0 units
  H-Head Production: 2.0 units
  H-Head Inventory: 39.0 units
Week 5:
  P-Head Production: 45.0 units
  P-Head Inventory: 36.0 units
  H-Head Production: 48.0 units
  H-Head Inventory: 39.0 units
Week 6:
  P-Head Production: 38.0 units
  P-Head Inventory: 29.0 units
  H-Head Production: 56.0 units
  H-Head Inventory: 47.0 units
Week 7:
  P-Head Production: 35.0 units
  P-Head Inventory: 28.0 units
  H-Head Production: 57.0 units
  H-Head Inventory: 46.0 units
Week 8:
  P-Head Production: 35.0 units
  P-Head Inventory